# Simulação de Reservatórios de Petróleo – 2021/2

## Trabalho 2

**Grupo:** Luan Rodrigues e Matheus Harduim.

**Professor:** Grazione de Souza.

**Instituição:** UERJ – IPRJ – DMC

### Desenvolvimento dos códigos e comentários.

Importando as bibliotecas do python que serão utilizadas

In [1]:
import numpy.matlib
import numpy as np
import math
import pprint
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Dados do problema
- $k = 10,0 × {10}^{−15} {m}^{2}$
- $φ = 0,25$
- ${S}_{w inicial} = 0,2$
- ${L}_{x} = 5000m$
- ${L}_{y} = 40m$
- ${L}_{z} = 10m$
- ${µ}_{w} = 1,0 × {10}^{−3}Pa·s$
- ${µ}_{o} = 0,8 × {10}^{−3}Pa·s$

In [2]:
k = 10.0* (10**-15)
phi = 0,25
Swinicial = 0,2
Lx = 5000
Ly = 40
Lz = 10
miw = 1.0*(10**-3)
mio = 0.8*(10**-3)
qt = 1.0*(10**-3)